In [1]:
# webbrowser : 브라우저로 특정 페이지를 여는 모듈
import webbrowser
# requests : 인터넷에서 파일과 웹 페이지를 다운로드 하는 모듈
import requests
# bs4 : HTML을 구문분석(parsing)하는 라이브러리
from bs4 import BeautifulSoup as BS
# selenium.webdriver : 웹 브라우저를 띄우고 제어하는 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
# urlopen으로 직접 열 수도 있는데.. requests로 다운받아놓는게 더 안전
from urllib.request import urlopen
import pyinputplus as pyip
from pathlib import Path

In [2]:
# webbroswer.open()함수 활용
# 주소의 법칙을 알면 pyinputplus 및 git bash(sys.argv[])와 연계 가능
findAddress = pyip.inputStr(prompt='찾는 주소를 입력해주세요.')
webbrowser.open(f'https://map.naver.com/p/search/{findAddress}?c=13.00,0,0,0,dh')

찾는 주소를 입력해주세요.

True

In [3]:
# request.get()함수는 다운로드할 url 문자열 → Response객체로 반환
res = requests.get('https://automatetheboringstuff.com/files/rj.txt')
# 다운로드 성공 여부 확인을 위해 res.raise_for_status() 메서드 활용
res.raise_for_status()
print(res.text[:250])

The Project Gutenberg EBook of Romeo and Juliet, by William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Projec


In [4]:
path = Path.cwd() / 'result_attachments' / 'walnut2'
# 이진파일 쓰기 모드로 열어야 → mode='wb'
with open(path / 'Romeo and Juliet.txt', mode='wb') as playFile:
    # res.iter_content() 메서드 이용하여 반복문을 돌 때 마다 chunk를 반환 → 각 chunk는 바이트 자료형
    for chunk in res.iter_content(100000):
        playFile.write(chunk)

In [5]:
res = requests.get('https://nostarch.com')
res.raise_for_status()
# html = urlopen(url)을 통해 res.text 대신 html 집어넣는 것도 가능하긴 한데..
# requests 모듈을 통해 Response 객체를 받은 후 text 속성을 이용하는 아래 방법이 더 좋은듯
bsObj = BS(res.text, 'html.parser')
type(bsObj)

bs4.BeautifulSoup

In [6]:
# BeautifulSoup 객체에서의 메서드들 → find, find_all, select(선택자 패턴) 등
# bsObj.select('div') → 이름이 div인 모든 요소
# bsObj.select('#author') → id 속성이 author인 요소
# bsObj.select('.notice') → CSS class 속성의 이름이 notice인 요소
# bsObj.select('div span') → <div> 요소 내 존재하는 요소 중 이름이 <span>인 모든 요소
# bsObj.select('input[name]') → 이름이 <input>인 요소 중 name 속성 값이 있는 모든 요소
# bsObj.select('input[type="button"]') → 이름이 <input>인 요소 중 type 속성 값이 button인 경우
# 다양한 선택자 패턴 결합 가능

In [7]:
len(bsObj.select('div')), type(bsObj.select('input[type="text"]')[0])

(340, bs4.element.Tag)

In [8]:
bsObj.select('input[type="text"]')

[<input class="form-control form-text" id="edit-keys-2" maxlength="128" name="keys_2" placeholder="Search" size="15" type="text" value=""/>,
 <input class="form-control form-text" id="edit-keys-2" maxlength="128" name="keys_2" placeholder="Search" size="15" type="text" value=""/>]

In [9]:
bsObj.select('input[type="text"]')[0].attrs, bsObj.select('input[type="text"]')[0].getText()

({'placeholder': 'Search',
  'class': ['form-control', 'form-text'],
  'type': 'text',
  'id': 'edit-keys-2',
  'name': 'keys_2',
  'value': '',
  'size': '15',
  'maxlength': '128'},
 '')

In [10]:
# 선택자 패턴으로 div 추출, text 메서드 활용
bsObj.select('div a')[0], bsObj.select('div a')[0].text.strip(), bsObj.select('div a')[0].getText().strip()

(<a class="element-invisible element-focusable" href="#main-content">Skip to main content</a>,
 'Skip to main content',
 'Skip to main content')

In [11]:
# get() 메서드를 활용하여 해당 속성 값을 반환
bsObj.select('#skip-link')[0], bsObj.select('#skip-link')[0].get('id'), bsObj.select('#skip-link')[0].attrs

(<div id="skip-link">
 <a class="element-invisible element-focusable" href="#main-content">Skip to main content</a>
 </div>,
 'skip-link',
 {'id': 'skip-link'})

In [12]:
# attrs 속성을 활용할 경우 키-값 형태의 딕셔너리로 속성 값 반환
bsObj.select('div')[0].get('id'), bsObj.select('div')[0].attrs

('skip-link', {'id': 'skip-link'})

In [13]:
# find_all()을 활용하여 해당 사이트의 책 이미지들 가져오기
# list comprehension과 간편 조건문 이용하여 책 제목들 추출
[x.get('title') for x in bsObj.find_all('img', {'class':'img-responsive'}) if x.get('title') is not None]

['Evading EDR cover',
 'Automate the Boring Stuff with Python, 2nd Edition',
 'How AI Works Cover',
 'Open Circuits Cover',
 'PowerShell for Sysadmins',
 'Engineering in Plain Sight Cover',
 'Evading EDR cover',
 'The Art of Machine Learning Cover',
 'Practical Julia Cover',
 'Arduino for Arduinians Cover',
 'How AI Works Cover',
 'The Android Malware Handbook cover',
 'Getting Started With FPGAs Cover',
 'Python Playground, 2nd Edition Cover',
 'The C# Type System cover',
 'Codebreaking cover',
 'Math for Security Cover',
 'The LEGO Lighting Book cover',
 'Hacks, Leaks, and Revelations cover',
 'Algorithmic Thinking, 2nd Edition',
 'JavaScript Crash Course Cover',
 'The Art of Randomness cover',
 'Modern Full Stack Development placeholder cover',
 'Locksport Cover',
 'Windows Security Internals with PowerShell cover',
 'Machine Learning and AI Beyond the Basics placeholder cover',
 'R Without Statistics placeholder cover',
 'PHP Crash Course Cover',
 'Writing a C Compiler Cover',
 'C+

In [14]:
bsObj.find_all('img', {'class':'img-responsive'}) == bsObj.select('img[class="img-responsive"]')

True

In [17]:
# Options() 및 add_argument() 활용하여 여러 옵션 추가 가능(by 업무 자동화 - bhban)
options = Options()
# add_argument 안에 'headless' 넣으면 헤드리스 자동화 가능
options.add_argument('--window-size=1600,900')
# 책 대신 원래 쓰던 방식 활용 → ChromeDriverManager().install()을 통해 직접 설치
driver = webdriver.Chrome(options=options)

In [18]:
# get() 메서드에 링크 주소 전달하여 컨트롤
driver.get('https://inventwithpython.com')
# page_source 속성으로 selenium 드라이버에서 page객체 생성 가능 → 이걸 통해 BeautifulSoup에서 활용 가능
page = driver.page_source
bsObj = BS(page, 'html.parser')

In [21]:
type(bsObj)

bs4.BeautifulSoup

In [23]:
bsObj.select('div a')[0].get('id')

'navbarDropdownReadMenuLink'

In [24]:
# 페이지에서 요소를 찾는 메서드로 driver.find_element 활용 가능
# By.CLASS_NAME, By.ID, By.CSS_SELECTOR, By.LINK_TEXT, By.NAME, By.TAG_NAME, By.XPATH 등
try:
    elems = driver.find_elements(By.CLASS_NAME, 'cover-thumb')
    print(elems[0].tag_name)
except:
    print('There is no elements.')

img


In [26]:
try:
    elems = driver.find_elements(By.CLASS_NAME, 'dropdown-toggle')
    print(elems[0].tag_name, elems[0].text, sep='\n')
except:
    print('There is no elements.')

a
Read for Free


In [28]:
for elem in elems:
    print(elem.text)

Read for Free
Buy Books
Udemy Courses


In [29]:
try:
    elems = driver.find_elements(By.CLASS_NAME, 'nav-link')
    print(elems[0].tag_name, elems[0].text, sep='\n')
except:
    print('There is no elements.')

a
Read for Free


In [30]:
for elem in elems:
    print(elem.text)

Read for Free
Buy Books
Udemy Courses
YouTube
Subreddit
Blog
Donate


In [31]:
elems = driver.find_elements(By.CLASS_NAME, 'cover-thumb')
# get_attribute() 메서드 이용하여 name 속성에 해당하는 값 추출
elems[1].get_attribute('alt')

'Cover of Python Programming Exercises, Gently Explained'

In [34]:
for elem in elems:
    print(elem.get_attribute('alt'))

Cover of Automate the Boring Stuff with Python
Cover of Python Programming Exercises, Gently Explained
Cover of The Recursive Book of Recusrion
Cover of The Big Book of Small Python Projects
Cover of Beyond the Basic Stuff with Python
Cover of Coding with Minecraft
Cover of Cracking Codes with Python
Cover of Invent Your Own Computer Games with Python
Cover of Scratch 3 Programming Playground
Cover of Making Games with Python & Pygame
Cover of Automate the Boring Stuff with Python
Cover of The Recursive Book of Recursion
Cover of Python Programming Exercises, Gently Explained
Cover of The Big Book of Small Python Projects
Cover of Beyond the Basic Stuff with Python
Cover of Coding with Minecraft
Cover of Cracking Codes with Python
Cover of Invent Your Own Computer Games with Python
Cover of Scratch 3 Programming Playground
Cover of Making Games with Python & Pygame


In [35]:
driver.find_element(By.LINK_TEXT, 'Read Online for Free')

<selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_305")>

In [43]:
try:
    # By.LINK_TEXT 활용하여 해당 텍스트가 있는 곳을 찾을 수가 있으며,
    linkElem = driver.find_elements(By.LINK_TEXT, 'Read Online for Free')
    # click() 메서드를 통해 클릭 가능
    linkElem[0].click()
except:
    print('There is no elements.')

In [44]:
linkElem

[<selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_305")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_322")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_425")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_426")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_427")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="CDB940DF246719E2DCDF194E85DFB5AB_element_428")>,
 <selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db

In [41]:
try:
    # By.PARTIAL_LINK_TEXT도 활용 가능
    linkElem = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Free to read')
    linkElem[0].click()
except:
    print('There is no elements.')

In [42]:
linkElem

[<selenium.webdriver.remote.webelement.WebElement (session="49043cf80c45f2c20aa8db45429b9113", element="C1469E3FB27B63F9C825266A66AA3BA9_element_33")>]

In [45]:
driver.get('https://login.metafilter.com')
# By.ID를 통해 id가 'user_name'인 것을 찾고
userElem = driver.find_element(By.ID, 'user_name')
# send_keys() 메서드를 통해 내용 입력
# id 등을 지울 때는 clear() 메서드를 활용하면 될듯
userElem.send_keys('id12345')
pwElem = driver.find_element(By.ID, 'user_pass')
pwElem.send_keys('password12345')
# submit() 메서드를 통해 로그인을 진행(로그인 버튼을 찾아서 클릭하는 걸 간단히 하는 개념)
pwElem.submit()

In [46]:
driver.get('https://login.metafilter.com')
# By.ID를 통해 id가 'user_name'인 것을 찾고
userElem = driver.find_element(By.NAME, 'user_name')
# send_keys() 메서드를 통해 내용 입력
# id 등을 지울 때는 clear() 메서드를 활용하면 될듯
userElem.send_keys('id12345')
pwElem = driver.find_element(By.NAME, 'user_pass')
pwElem.send_keys('password12345')
# submit() 메서드를 통해 로그인을 진행(로그인 버튼을 찾아서 클릭하는 걸 간단히 하는 개념)
pwElem.submit()

In [49]:
driver.back()

In [50]:
driver.get('https://nostarch.com')
# By.TAG_NAME을 'html'로 설정하여 사이트 자체를 컨트롤할 수 있게 만들고
elem = driver.find_element(By.TAG_NAME, 'html')
# 'Keys.'를 활용한 특수 키를 지정하여 원하는 방법으로 진행
# Keys.DOWN, Keys.UP, Keys.LEFT, Keys.RIGHT, Keys.ENTER, Keys.RETURN, Keys.HOME, Keys.END, Keys.PAGE_DOWN, Keys.PAGE_UP
# Keys.ESCAPE(ESC키), Keys.BACK_SPACE, Keys.DELETE, Keys.TAB 등
elem.send_keys(Keys.END)

In [51]:
driver.quit()